# Import, Generic Setup

In [1]:
import papermill as pm
from itertools import product
import sys
import os

In [2]:
template_dir = '~/Documents/GitHub/StructuredPredictionError/papermill_templates/'
wdir = '~/Documents/GitHub/StructuredPredictionError/papermill_templates/output/'

In [3]:
def product_dict(**kwargs):
    keys = kwargs.keys()
    vals = kwargs.values()
    for instance in product(*vals):
        yield dict(zip(keys, instance))

In [4]:
def get_inputfp(model, folder_name):
    input_nb = model + "Template.ipynb"
    indir = wdir + model + '/' + folder_name + '/'

    inputfp = os.path.expanduser(template_dir) + input_nb
    return indir, inputfp

In [5]:
def run_nbs(indir, inputfp, params, start_idx=0):
#     if model == 'RL':
#         params = product_dict(**rl_dict)
#         # input_nb = 'RLTemplate.ipynb'
#         # indir = wdir + 'RL/exp_kern_multik/'
#     elif model == 'RF':
#         params = product_dict(**rf_dict)
#         # input_nb = 'RFTemplate.ipynb'
#         # indir = wdir + 'RF/exp_kern_multik/'
#     elif model == 'RFRefits':
#         params = product_dict(**rfrefits_dict)
#         # input_nb = 'RFRefitsTemplate.ipynb'
#         # indir = wdir + 'RFRefits/exp_kern_multik/'

#     input_nb = model + "Template.ipynb"
#     indir = wdir + model + '/' + folder_name + '/'

#     inputfp = os.path.expanduser(template_dir) + input_nb
    if not os.path.exists(inputfp):
        os.makedirs(inputfp)

#     output_nb = 'output.ipynb'
#     output_nb = "\u03B4{delta}_kern{kernel}_nls{noise_length_scale}_nv{noise_nu}_Xls{X_length_scale}_Xv{X_nu}"

    for i,param in enumerate(params):
        output_nb = f"\u03B4{param['delta']}_snr{param['snr']}" \
                    f"_nk{param['noise_kernel']}" \
                    f"_nls{param['noise_length_scale']}" \
                    f"_nv{param['noise_nu']}_Xk{param['X_kernel']}" \
                    f"_Xls{param['X_length_scale']}_Xv{param['X_nu']}"
        if 'alpha' in param:
            output_nb = f"\u03B1={param['alpha']}_" + output_nb
            
#         outputdir = os.path.expanduser(indir) + 'run_' + str(i+start_idx) + '/'
        outputdir = os.path.expanduser(indir) + 'run_' \
                    + str(i+start_idx) + '_' + output_nb + '/'
        output_nb += ".ipynb"
        outputfp = outputdir + output_nb
        if not os.path.exists(outputdir):
            os.makedirs(outputdir)
        param['savedir'] = outputdir
        param['idx'] = i
        pm.execute_notebook(
            inputfp,
            outputfp,
            parameters=param
        )

# Compare RL: Correction vs CV

In [6]:
rl_dict = {
    'niter': [50],
    'n': [400],
    'p': [500],
    's': [30],
    'k': [10],
    'snr': [0.4],
    'lambd': [1.],
    'delta':[0.8],
    'alpha': [0.05],
    'nboot': [100],
    'noise_kernel': ['matern'],
    'noise_length_scale':[1.],
    'noise_nu':[0.5, 2.5],
    'X_kernel': ['matern'],
    'X_length_scale':[1.],
    'X_nu':[0.5, 2.5],
}

params = product_dict(**rl_dict)
indir, inputfp = get_inputfp('RL', 'RLWR_our_vs_CV_alpha05')
run_nbs(indir, inputfp, params)

Executing:   0%|          | 0/18 [00:00<?, ?cell/s]

Executing:   0%|          | 0/18 [00:00<?, ?cell/s]

Executing:   0%|          | 0/18 [00:00<?, ?cell/s]

Executing:   0%|          | 0/18 [00:00<?, ?cell/s]

# Compare RL corr $Y, Y^*$: correction vs CV 

In [7]:
rl_dict = {
    'niter': [50],
    'n': [400],
    'p': [500],
    's': [30],
    'k': [10],
    'snr': [0.4],
    'lambd': [1.],
    'delta':[0.8],
    'alpha': [0.05],
    'nboot': [100],
    'noise_kernel': ['matern'],
    'noise_length_scale':[1.],
    'noise_nu':[0.5, 2.5],
    'X_kernel': ['matern'],
    'X_length_scale':[1.],
    'X_nu':[0.5, 2.5],
}

params = product_dict(**rl_dict)
indir, inputfp = get_inputfp('CRL', 'CRLWR_our_vs_CV_alpha05')
run_nbs(indir, inputfp, params, 48)

Executing:   0%|          | 0/21 [00:00<?, ?cell/s]

Executing:   0%|          | 0/21 [00:00<?, ?cell/s]

Executing:   0%|          | 0/21 [00:00<?, ?cell/s]

Executing:   0%|          | 0/21 [00:00<?, ?cell/s]

# Compare VRF: Correction vs CV

In [17]:
rf_dict = {
    'niter': [50],
    'n': [900],
    'p': [30],
    's': [30],
    'k': [10],
    'snr': [0.4],
    'n_estimators': [100],
    'max_depth': [6],
    'delta':[0.8],
    'alpha': [0.05],
    'nboot': [100],
    'noise_kernel': ['matern'],
    'noise_length_scale':[1.],
    'noise_nu':[0.5, 2.5],
    'X_kernel': ['matern'],
    'X_length_scale':[1.],
    'X_nu':[0.5, 2.5],
}

params = product_dict(**rf_dict)
indir, inputfp = get_inputfp('RF', 'RF_our_vs_CV_alpha05')
run_nbs(indir, inputfp, params)

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

# Compare VRF corr $Y, Y^*$: correction vs CV 

# Compare BF full refit: Correction vs CV

In [18]:
rf_dict = {
    'niter': [50],
    'n': [900],
    'p': [30],
    's': [30],
    'k': [10],
    'snr': [0.4],
    'n_estimators': [100],
    'max_depth': [6],
    'delta':[0.8],
    'noise_kernel': ['matern'],
    'noise_length_scale':[1., 5.],
    'noise_nu':[0.5, 2.5],
    'X_kernel': [None, 'matern'],
    'X_length_scale':[1., 5.],
    'X_nu':[0.5, 2.5],
}

params = product_dict(**rf_dict)
indir, inputfp = get_inputfp('BF', 'BFFR_our_vs_CV')
run_nbs(indir, inputfp, params)

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

...

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

# Compare BF full refit, corr $Y, Y^*$: Correction vs CV

# Compare: VRF vs WRF vs BFWR vs BFFR

In [7]:
rf_dict = {
    'niter': [50],
    'n': [900],
    'p': [30],
    's': [30],
    'snr': [0.4],
    'n_estimators': [5],
    'max_depth': [6],
    'delta':[0.8],
    'noise_kernel': ['matern'],
    'noise_length_scale':[1.],
    'noise_nu':[0.5, 2.5],
    'X_kernel': ['matern'],
    'X_length_scale':[1.],
    'X_nu':[0.5, 2.5],
}

params = product_dict(**rf_dict)
indir, inputfp = get_inputfp('RFBF', 'RFBF_comps_iid_5trees')
run_nbs(indir, inputfp, params)

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

In [8]:
rf_dict = {
    'niter': [50],
    'n': [900],
    'p': [30],
    's': [30],
    'snr': [0.4],
    'n_estimators': [100],
    'max_depth': [6],
    'delta':[0.8],
    'noise_kernel': ['matern'],
    'noise_length_scale':[1.],
    'noise_nu':[0.5, 2.5],
    'X_kernel': ['matern'],
    'X_length_scale':[1.],
    'X_nu':[0.5, 2.5],
}

params = product_dict(**rf_dict)
indir, inputfp = get_inputfp('RFBF', 'RFBF_comps_new')
run_nbs(indir, inputfp, params)

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

# Compare BF: OLS vs GLS